## POS Tagging

Tenemos que mapear en python las etiquetas de la página: http://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/french-tagset.html para que cuando aparezcan sean reemplazados por las etiquetas que correspondan del tagset Upenn (se muestran abajo).

In [5]:
import nltk
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

## TreeTagger

Para que funcione la librería hay que seguir el proceso que se muestra en esta web: http://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/

Todos los ficheros deben estar en el mismo directorio.La clave es que los ficheros que lo necesiten no tengan sin descomprimir el .gz. De todos modos adjunto los ficheros preparaditos para ejecutar el script de instalar _sh install-tagger.sh_.

Para que tire el diccionario francés meter en TreeTagger/lib el fichero _french-utf8.par_.

## Python3 Tree Tagger


Clonar el repositorio de este pollo. https://github.com/miotto/treetagger-python

##### NOTA: Directamente adjunto el fichero python para no clonar ni nada

No instalar como dice él con setup.py (al menos a mí no me lo reconoce el notebook despúes). Mejor importar directamente la librería treetagger.py como toda la vida.

Recordar añadir con export el PATH al ejecutable como se dice en la instalación



## Pruebas

El programa treetagger se es capaz de ejecutar los procedimientos de 'tokenizing' y de 'stemming' sobre el corpus objetivo. Posteriormente cogeremos el resultado y los mapearemos al tagset Upenn y para dárselo al NER de NLTK.

In [22]:
from treetagger import TreeTagger
tt = TreeTagger(language='french')
tt.tag('Elle est jolie')

test_ttg = tt.tag('Emmanuel Macron a gagné à la candidate du Front National, Marine Le Pen, par un soixante-cinq percent des votes dans les elections de la France.')
print(test_ttg)

[['Emmanuel', 'NAM', 'Emmanuel'], ['Macron', 'NAM', '<unknown>'], ['a', 'VER:pres', 'avoir'], ['gagné', 'VER:pper', 'gagner'], ['à', 'PRP', 'à'], ['la', 'DET:ART', 'le'], ['candidate', 'NOM', 'candidat'], ['du', 'PRP:det', 'du'], ['Front', 'NAM', '<unknown>'], ['National', 'NAM', '<unknown>'], [',', 'PUN', ','], ['Marine', 'NAM', 'Marine'], ['Le', 'DET:ART', 'le'], ['Pen', 'NAM', '<unknown>'], [',', 'PUN', ','], ['par', 'PRP', 'par'], ['un', 'DET:ART', 'un'], ['soixante-cinq', 'NUM', 'soixante-cinq'], ['percent', 'VER:pres', 'percer'], ['des', 'PRP:det', 'du'], ['votes', 'NOM', 'vote'], ['dans', 'PRP', 'dans'], ['les', 'DET:ART', 'le'], ['elections', 'NOM', '<unknown>'], ['de', 'PRP', 'de'], ['la', 'DET:ART', 'le'], ['France', 'NAM', 'France'], ['.', 'SENT', '.']]


A continuación se carga el diccionario encargado de mapear el etiquetado del formato empleado por TreeTagger al formato UPenn.

In [23]:
#Dictionary uses following pair convention --> 'Treetagger_french_tagging':'UPenn_tagging'
mapping = {'ABR':'CC',
       'ADJ':'JJ',
       'ADV':'RB',
       'DET:ART':'DT',
       'DET:POS':'PRP$',
       'INT':'UH',
       'KON':'CC',
       'NAM':'NNP',
       'NOM':'NN',
       'NUM':'CD',
       'PRO':'PRP',
       'PRO:DEM':'DT',
       'PRO:IND':'DT',
       'PRO:PER':'PRP',
       'PRO:POS':'PRP$',
       'PRO:REL':'DT',
       'PRP':'IN',
       'PRP:det':'RP',
       'PUN':'SYM',
       'PUN:cit':'SYM',
       'SENT':'.',
       'SYM':'SYM',
       'VER:cond':'MD',
       'VER:futu':'MD',
       'VER:impe':'VBP',
       'VER:impf':'VBD',
       'VER:infi':'VB',
       'VER:pper':'VBN',
       'VER:ppre':'VBG',
       'VER:pres':'VB',
       'VER:simp':'VBD',
       'VER:subi':'VBN',
       'VER:subp':'VBD',      
}

Ejecutamos el mapeado sobre el corpus resultante de TreeTagger.

In [24]:
test_mapped=[]
j=0
for i in test_ttg:
    if i[1] in mapping:
        test_mapped.append((test_ttg[j][0],mapping[i[1]]))
    j+=1
print(test_mapped)

[('Emmanuel', 'NNP'), ('Macron', 'NNP'), ('a', 'VB'), ('gagné', 'VBN'), ('à', 'IN'), ('la', 'DT'), ('candidate', 'NN'), ('du', 'RP'), ('Front', 'NNP'), ('National', 'NNP'), (',', 'SYM'), ('Marine', 'NNP'), ('Le', 'DT'), ('Pen', 'NNP'), (',', 'SYM'), ('par', 'IN'), ('un', 'DT'), ('soixante-cinq', 'CD'), ('percent', 'VB'), ('des', 'RP'), ('votes', 'NN'), ('dans', 'IN'), ('les', 'DT'), ('elections', 'NN'), ('de', 'IN'), ('la', 'DT'), ('France', 'NNP'), ('.', '.')]


Una vez tenemos nuestro corpus etiquetado en formato UPenn, empleamos la librería ne_chunk de NLTK para realizar el  procesamiento NER.

In [25]:
from nltk import ne_chunk
ne_tagged = ne_chunk(test_mapped)
print(ne_tagged)

(S
  (PERSON Emmanuel/NNP)
  Macron/NNP
  a/VB
  gagné/VBN
  à/IN
  la/DT
  candidate/NN
  du/RP
  (PERSON Front/NNP National/NNP)
  ,/SYM
  (PERSON Marine/NNP Le/DT Pen/NNP)
  ,/SYM
  par/IN
  un/DT
  soixante-cinq/CD
  percent/VB
  des/RP
  votes/NN
  dans/IN
  les/DT
  elections/NN
  de/IN
  la/DT
  (GPE France/NNP)
  ./.)


Como podemos ver la librería NER de NLTK es capaz de reconocer a dos personas (Macron y Le Pen) y un lugar (Francia) correctamente. También ha sido capaz de reconocer el partido Front National pero como si se tratase de una persona en vez de una organización. Por último, comentar que en este caso no ha sido capaz de reconocer el nombre 'Macron' como parte de la entidad persona 'Emmanuel'.

El siguiente apartado simplemente es una muestra de ejemplo en la que el mapeo se ha hecho de forma manual.

In [9]:
test = [('Emmanuel','NNP'),('Macron','NNP'),('est','VBG'),('le','DT'),('nouveau','JJ'),('président','NN'),('de','IN'),('la','DT'),('France','NNP')]
ne_tagged = ne_chunk(test)
print(ne_tagged)

(S
  (PERSON Emmanuel/NNP)
  (PERSON Macron/NNP)
  est/VBG
  le/DT
  nouveau/JJ
  président/NN
  de/IN
  la/DT
  (GPE France/NNP))
